Обработка естественной речи
============================================================

**Key words:** 
   nlp, spacy, word2vec


<h3> Plan </h3>
  * ** Напоминание возможных постановок задач nlp ** (10 minutes)
  * ** Word2vec ** (30 minutes)
  * ** Предобработка при помощи SpaCy ** (30 minutes)
  * ** Примеры задач nlp *** (20 minutes)</span> 

# Word2vec

По мотивам http://nlpx.net/archives/179 и https://github.com/danielfrg/word2vec

word2vec — это инструмент (набор алгоритмов) для расчета векторных представлений слов, реализует две основные архитектуры — **Continuous Bag of Words (CBOW)** и **Skip-gram**. На вход подается корпус текста, а на выходе получается набор векторов слов.

Более формально задача стоит так: **максимизация косинусной близости** между векторами слов (скалярное произведение векторов), которые появляются рядом друг с другом, и минимизация косинусной близости между векторами слов, которые не появляются друг рядом с другом. Рядом друг с другом в данном случае значит в близких контекстах.

**Напоминание:** косинусная близость
    $$\text{similarity} = \cos(\theta) = {A \cdot B \over \|A\| \|B\|} = \frac{ \sum\limits_{i=1}^{n}{A_i \times B_i} }{ \sqrt{\sum\limits_{i=1}^{n}{(A_i)^2}} \times \sqrt{\sum\limits_{i=1}^{n}{(B_i)^2}} }$$

word2vec это НЕ глубокое обучение. Потому как там применяется обычная, а не «глубокая» нейросеть прямого распространения (Feed-forward Neural Network)

Вообще, CBOW и Skip-gram — это нейросетевые архитектуры, которые описывают, как именно нейросеть «учится» на данных и «запоминает» представления слов. Принципы у обоих архитектур разные. Принцип работы CBOW — предсказывание слова при данном контексте, а skip-gram наоборот — предсказывается контекст при данном слове.

<img src="./word2vec.png">

- **Continuous Bag of Words (CBOW)** – обычная модель мешка слов с учётом четырёх ближайших соседей термина (два предыдущих и два последующих слова) без учёта порядка следования.

- **k-skip-n-gram** — это последовательность длиной n, где элементы находятся на расстоянии не более, чем k друг от друга. 

## Негативное сэмплирование

Задача построения модели word2vec: максимизация близости векторов слов (скалярное произведение векторов), которые появляются рядом друг с другом, и минимизация близости векторов слов, которые не появляются друг рядом с другом.

Представим упрощенное уравнение этой идеи:
    $$
        \frac{v_c\times v_t}{\sum v_{c1}\times v_t}
    $$

В числителе мы имеем близость слов контекста ($v_{c}$) и целевого слова ($v_{t}$), в знаменателе — близость всех других контекстов ($v_{c1}$) и целевого слова ($v_{t}$). Проблема в том, что считать все это долго и сложно. Негативное сэмплирование: мы не считаем ВСЕ возможные контексты, а выбираем случайным образом НЕСКОЛЬКО контекстов $v_{c1}$. Такой подход значительно облегчает процесс тренировки word2vec.

## Немного практики

In [1]:
%load_ext autoreload
%autoreload 2

### Обучение

Скачаем данные: [http://mattmahoney.net/dc/text8.zip](http://mattmahoney.net/dc/text8.zip)

In [2]:
import word2vec

In [3]:
word2vec.word2phrase('text8', 'text8-phrases', verbose=True)

Starting training using file text8
Words processed: 17000K     Vocab size: 4399K  rocessed: 2800K     Vocab size: 1104K  
Vocab size (unigrams + bigrams): 2419827
Words in train file: 17005206


Создаем файл `text8-phrases`, чтобы подать его на вход  `word2vec`. Этот шаг можно пропустить и подать на вход сырые данные.

Обучаем word2vec модель.

In [4]:
word2vec.word2vec('text8-phrases', 'text8.bin', size=100, verbose=True)

Starting training using file text8-phrases
Vocab size: 98331
Words in train file: 15857306


Alpha: 0.003270  Progress: 86.93%  Words/

Alpha: 0.000002  Progress: 100.03%  Words/thread/sec: 265.00k  s/thread/sec: 264.78k  : 0.002620  Progress: 89.53%  Words/thread/sec: 264.81k  Words/thread/sec: 264.74k  pha: 0.002187  Progress: 91.27%  Words/thread/sec: 264.79k  %  Words/thread/sec: 264.75k  .86%  Words/thread/sec: 264.80k  k   95.58%  Words/thread/sec: 264.78k  .79k  ss: 97.31%  Words/thread/sec: 264.67k  264.82k  gress: 99.08%  Words/thread/sec: 264.79k  c: 264.98k  

Файл `text8.bin` содержит вектора слов в бинарном виде.

Создаем кластеры на основе обученной модели.

In [5]:
word2vec.word2clusters('text8', 'text8-clusters.txt', 100, verbose=True)

Starting training using file text8
Vocab size: 71291
Words in train file: 16718843


Alpha: 0.0030

Alpha: 0.000002  Progress: 100.03%  Words/thread/sec: 263.34k  2619  Progress: 89.53%  Words/thread/sec: 262.65k  /thread/sec: 262.73k  0.002215  Progress: 91.15%  Words/thread/sec: 262.56k  rds/thread/sec: 262.68k  a: 0.001811  Progress: 92.77%  Words/thread/sec: 262.80k   Words/thread/sec: 262.89k  lpha: 0.001405  Progress: 94.39%  Words/thread/sec: 262.92k  0%  Words/thread/sec: 262.89k   6.82%  Words/thread/sec: 263.05k  7k  : 98.46%  Words/thread/sec: 263.15k  3.31k  

Файл `text8-clusters.txt` содержит кластеры для каждого слова словаря.

### Предсказания

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import word2vec

In [8]:
model = word2vec.load('text8.bin')

Словарь:

In [9]:
model.vocab

array(['</s>', 'the', 'of', ..., 'dakotas', 'nias', 'burlesques'],
      dtype='<U78')

Вся матрица:

In [10]:
model.vectors.shape

(98331, 100)

In [11]:
model.vectors

array([[ 0.14333282,  0.15825513, -0.13715845, ...,  0.05456942,
         0.10955409,  0.00693387],
       [ 0.10136448, -0.01455281,  0.10195114, ..., -0.10148882,
         0.12464498, -0.02213244],
       [ 0.18440248,  0.03617   ,  0.22163972, ...,  0.0482553 ,
         0.09393772,  0.08885094],
       ...,
       [-0.10395487,  0.04674925,  0.12876071, ...,  0.12327843,
        -0.09347397, -0.05557026],
       [ 0.02035719,  0.03220995,  0.09305181, ...,  0.03481731,
        -0.19664939, -0.12811995],
       [-0.00415086, -0.04732238,  0.04623443, ...,  0.04687344,
        -0.11005756, -0.07075066]])

Мы можем узнать вектор любого слова из словаря

In [12]:
model['dog'].shape

(100,)

In [13]:
model['dog'][:10]

array([ 0.11632952,  0.01694006,  0.08824241, -0.0306368 ,  0.15153386,
       -0.05756098,  0.00574881,  0.09358197,  0.10525652, -0.07909904])

Мы можем посчитать попарные расстояния между двумя или несколькими словами

In [14]:
model.distance("dog", "cat", "fish")

[('dog', 'cat', 0.8793564673119473),
 ('dog', 'fish', 0.5833656659468412),
 ('cat', 'fish', 0.6177639537413828)]

## Сходство

In [15]:
indexes, metrics = model.similar("dog")
indexes, metrics

(array([ 2437,  5478,  7593, 10309,  2428,  2670,  2391,  3964,  4812,
        10230]),
 array([0.87935647, 0.84306434, 0.7819305 , 0.77183196, 0.76222532,
        0.75403152, 0.75380452, 0.75322427, 0.75230246, 0.7505155 ]))

На выходе получаем два списка:
1. список индексов слов, схожих с данным
2. список значений сходства

In [16]:
model.vocab[indexes]

array(['cat', 'cow', 'goat', 'rat', 'bear', 'bird', 'girl', 'dogs',
       'wolf', 'pig'], dtype='<U78')

In [17]:
model.generate_response(indexes, metrics)

rec.array([('cat', 0.87935647), ('cow', 0.84306434), ('goat', 0.7819305 ),
           ('rat', 0.77183196), ('bear', 0.76222532),
           ('bird', 0.75403152), ('girl', 0.75380452),
           ('dogs', 0.75322427), ('wolf', 0.75230246),
           ('pig', 0.7505155 )],
          dtype=[('word', '<U78'), ('metric', '<f8')])

In [18]:
model.generate_response(indexes, metrics).tolist()

[('cat', 0.8793564673119473),
 ('cow', 0.8430643421426257),
 ('goat', 0.7819305043099686),
 ('rat', 0.7718319569153317),
 ('bear', 0.7622253242697923),
 ('bird', 0.7540315175337027),
 ('girl', 0.7538045170919909),
 ('dogs', 0.75322426644677),
 ('wolf', 0.7523024639778775),
 ('pig', 0.7505155012900172)]

### Фразы

Поскольку мы обучили алгоритм на `word2phrase` мы можем отобразить схотство исходя из "фраз".

In [19]:
indexes, metrics = model.similar('los_angeles')
model.generate_response(indexes, metrics).tolist()

[('san_francisco', 0.893406648007967),
 ('san_diego', 0.8728984083777882),
 ('las_vegas', 0.8418905944462175),
 ('miami', 0.8313320556508131),
 ('seattle', 0.8303476300600032),
 ('california', 0.8274332044660233),
 ('cleveland', 0.8266549604180308),
 ('detroit', 0.8210469809650927),
 ('chicago', 0.8184177165262507),
 ('cincinnati', 0.8164385911935804)]

### Аналогии и линейность

 <img src="./pict.jpg">
 
 `king - man + woman = queen` 

In [20]:
indexes, metrics = model.analogy(pos=['king', 'woman'], neg=['man'])
indexes, metrics

(array([1087, 7523, 1145, 1335, 1827, 3141, 6768,  648, 8419, 4980]),
 array([0.29351067, 0.27462383, 0.27178875, 0.26943102, 0.2651588 ,
        0.264542  , 0.2640163 , 0.26312415, 0.26281856, 0.25769584]))

In [21]:
model.generate_response(indexes, metrics).tolist()

[('queen', 0.293510674759427),
 ('empress', 0.2746238293668638),
 ('prince', 0.2717887471461545),
 ('wife', 0.26943102115434225),
 ('throne', 0.2651588009936282),
 ('monarch', 0.2645420049766334),
 ('regent', 0.26401629581670216),
 ('emperor', 0.2631241459676974),
 ('aragon', 0.262818561874043),
 ('heir', 0.25769583841034893)]

### Кластеры

In [22]:
clusters = word2vec.load_clusters('text8-clusters.txt')

In [23]:
clusters.vocab

array(['</s>', 'the', 'of', ..., 'bredon', 'skirting', 'santamaria'],
      dtype='<U29')

Мы можем посмотреть все слова, находящиеся в определенном кластере

In [29]:
clusters.get_words_on_cluster(90).shape

(205,)

In [25]:
clusters.get_words_on_cluster(90)[:10]

array(['along', 'associated', 'working', 'relations', 'relationship',
       'deal', 'compared', 'combined', 'contrast', 'contact'],
      dtype='<U29')

Мы можем добавить кластеры в модель word2vec и генерироваь ответы, включающие кластеры

In [26]:
model.clusters = clusters

In [27]:
indexes, metrics = model.analogy(pos=["paris", "germany"], neg=["france"])

In [28]:
model.generate_response(indexes, metrics).tolist()

[('berlin', 0.32275486206528214, 15),
 ('munich', 0.2862855188930788, 21),
 ('vienna', 0.2760421477296262, 12),
 ('st_petersburg', 0.2712241488630337, 61),
 ('leipzig', 0.2692546127699573, 8),
 ('moscow', 0.26664249371504267, 74),
 ('dresden', 0.2546390773975281, 71),
 ('prague', 0.24980033046195801, 72),
 ('z_rich', 0.24818770928729006, 80),
 ('bonn', 0.24179059607436923, 77)]

**Еще несколько комментариев:**
    
- CBOW работает быстрее, зато Skip-gram работает лучше, особенно для относительно редких слов
- Иерархический софтмакс хорошо подходит для создания лучшей модели относительно редких слов, негативное сэмплирование же лучше моделирует более частотные слова.
- Применение суб-сэмплирования улучшает производительность. Рекомендуемый параметр субсэмплирования от 1e-3 до 1е-5
- Размер вектора чем больше, тем лучше (впрочем, не всегда, это от корпуса зависит)
- Размер окна — для Skip-gram оптимальный размер около 10, для CBOW — в районе 5

**Одна из работающих схем:**

skip-gram + negative sampling + окно 10 слов + субсэмплирование 1е-5 + размер вектора 300

# Предобработка при помощи spacy

SpaCy это Cython/Python библиотека для обработки текстов. Работает быстрее чем NLTK, имеет довольно широкие возможности. https://spacy.io

In [32]:
import spacy

nlp = spacy.load('en_core_web_sm')
doc = nlp(u'Apple is looking at buying U.K. startup for $1 billion')
for token in doc:
    print(token.lemma_)

apple
be
look
at
buy
u.k.
startup
for
$
1
billion


### Определение стоп-слов, парсинг зависимостей

- Text: The original word text.
- Lemma: The base form of the word.
- POS: The simple part-of-speech tag.
- Tag: The detailed part-of-speech tag.
- Dep: Syntactic dependency, i.e. the relation between tokens.
- Shape: The word shape – capitalisation, punctuation, digits.
- is alpha: Is the token an alpha character?
- is stop: Is the token part of a stop list, i.e. the most common words of the language?

In [33]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(u'Apple is looking at buying U.K. startup for $1 billion')

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop)

Apple apple PROPN NNP nsubj Xxxxx True False
is be VERB VBZ aux xx True True
looking look VERB VBG ROOT xxxx True False
at at ADP IN prep xx True True
buying buy VERB VBG pcomp xxxx True False
U.K. u.k. PROPN NNP compound X.X. False False
startup startup NOUN NN dobj xxxx True False
for for ADP IN prep xxx True True
$ $ SYM $ quantmod $ False False
1 1 NUM CD compound d False False
billion billion NUM CD pobj xxxx True False


In [34]:
from spacy import displacy

displacy.render(doc, style='dep', jupyter=True)

In [35]:
options = {'compact': True, 'bg': '#09a3d5',
           'color': 'white', 'font': 'Source Sans Pro'}
displacy.render(doc, style='dep', options=options, jupyter=True)

### Распознавание именованных сущностей

In [5]:
displacy.render(doc, style='ent', jupyter=True)

In [6]:
colors = {'ORG': 'linear-gradient(90deg, #aa9cfc, #fc9ce7)'}
options = {'ents': ['ORG'], 'colors': colors}
displacy.render(doc, style='ent', options=options, jupyter=True)

Еще пример

In [7]:
text = """But Google is starting from behind. The company made a late push
into hardware, and Apple’s Siri, available on iPhones, and Amazon’s Alexa
software, which runs on its Echo and Dot devices, have clear leads in
consumer adoption."""

nlp = spacy.load('en_core_web_sm')
doc = nlp(text)
displacy.render(doc, style='ent', jupyter=True)

*Многовато ошибок*

### Оценка сходства

In [8]:
nlp = spacy.load('en_core_web_md')  # make sure to use larger model!
tokens = nlp(u'dog cat banana')

for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))

dog dog 1.0
dog cat 0.0
dog banana 0.0
cat dog 0.0
cat cat 1.0
cat banana -0.044681177
banana dog -7.828739e+17
banana cat -8.242222e+17
banana banana 1.0


In [9]:
tokens = nlp(u'Apple is looking at buying U.K. startup for $1 billion')

for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))

Apple Apple 1.0
Apple is -0.05732172
Apple looking -2.8055505e-21
Apple at 0.0
Apple buying -8.315521e+17
Apple U.K. 0.042299896
Apple startup -2.2416173e-21
Apple for -0.057875555
Apple $ -0.036178503
Apple 1 -9.8122005e+17
Apple billion 1.8286364e-21
is Apple 0.0
is is 1.0
is looking -1.3928241e+18
is at 0.0
is buying -3.56524e-21
is U.K. 0.061713193
is startup 0.060328145
is for 0.0
is $ 9.736638e+17
is 1 -0.07760425
is billion -9.078322e+17
looking Apple -2.8055505e-21
looking is -1.3928241e+18
looking looking 1.0
looking at -3.2815228e-21
looking buying 0.0
looking U.K. -1.0278183e+18
looking startup 0.0
looking for -0.076234676
looking $ -2.5833796e-21
looking 1 -0.070065476
looking billion 0.044432882
at Apple 0.0
at is -1.2367909e+18
at looking 0.0
at at 1.0
at buying 0.0
at U.K. 0.0
at startup 8.921919e+17
at for -0.06769437
at $ 0.0
at 1 1.1476879e+18
at billion 0.0
buying Apple -8.315521e+17
buying is -0.06576707
buying looking 0.0
buying at 0.0
buying buying 1.0
buying U.K.

In [10]:
nlp = spacy.load('en_core_web_md')
doc = nlp(u"Apple and banana are similar. Pasta and hippo aren't.")

apple = doc[0]
banana = doc[2]
pasta = doc[6]
hippo = doc[8]

print('apple <-> banana', apple.similarity(banana))
print('pasta <-> hippo', pasta.similarity(hippo))
print(apple.has_vector, banana.has_vector, pasta.has_vector, hippo.has_vector)

apple <-> banana -7.7179016e+17
pasta <-> hippo -0.04038117
True True True True


# Еще несколько примеров задач

### Удаление имен

In [41]:
# Загрузка английской NLP-модели
nlp = spacy.load('en_core_web_lg')
 
# Если токен является именем, заменяем его словом "REDACTED" 
def replace_name_with_placeholder(token):
    if token.ent_iob != 0 and token.ent_type_ == "PERSON":
        return "[REDACTED] "
    else:
        return token.string
 
 # Проверка всех сущностей
def scrub(text):
    doc = nlp(text)
    for ent in doc.ents:
        ent.merge()
    tokens = map(replace_name_with_placeholder, doc)
    return "".join(tokens)
 
s = """
In 1950, Alan Turing published his famous article "Computing Machinery and Intelligence". In 1957, Noam Chomsky’s 
Syntactic Structures revolutionized Linguistics with 'universal grammar', a rule based system of syntactic structures.
"""
 
print(scrub(s))


In 1950, [REDACTED] published his famous article "Computing Machinery and Intelligence". In 1957, [REDACTED] 
Syntactic Structures revolutionized Linguistics with 'universal grammar', a rule based system of syntactic structures.



### Извлечение фактов

In [42]:
import textacy.extract
 
# Загрузка английской NLP-модели
nlp = spacy.load('en_core_web_lg')
 
# Текст для анализа
text = """London is the capital and most populous city of England and  the United Kingdom.  
Standing on the River Thames in the south east of the island of Great Britain, 
London has been a major settlement  for two millennia.  It was founded by the Romans, 
who named it Londinium.
"""
 
# Анализ
doc = nlp(text)
 
# Извлечение полуструктурированных выражений со словом London
statements = textacy.extract.semistructured_statements(doc, "London")
 
# Вывод результатов
print("Here are the things I know about London:")
 
for statement in statements:
    subject, verb, fact = statement
    print(f" - {fact}")

Here are the things I know about London:
 - the capital and most populous city of England and  the United Kingdom.  

 - a major settlement  for two millennia.  


### Автодополнение

In [43]:
import textacy.extract
 
# Загрузка английской NLP-модели
nlp = spacy.load('en_core_web_lg')
 
# Текст для анализа
text = """London is the capital and most populous city of England and  the United Kingdom.  
Standing on the River Thames in the south east of the island of Great Britain, 
London has been a major settlement  for two millennia.  It was founded by the Romans, 
who named it Londinium.
London is the capital and most populous city of England and  the United Kingdom.  
Standing on the River Thames in the south east of the island of Great Britain, 
London has been a major settlement  for two millennia.  It was founded by the Romans, 
who named it Londinium.
London is the capital and most populous city of England and  the United Kingdom.  
Standing on the River Thames in the south east of the island of Great Britain, 
London has been a major settlement  for two millennia.  It was founded by the Romans, 
who named it Londinium."""
 
# Анализ
doc = nlp(text)
 
# Извлечение фрагментов
noun_chunks = textacy.extract.noun_chunks(doc, min_freq=3)
 
# Перевод в нижний регистр
noun_chunks = map(str, noun_chunks)
noun_chunks = map(str.lower, noun_chunks)
 
# вывод всех фрагментов, состоящих из 2 слов и более
for noun_chunk in set(noun_chunks):
    if len(noun_chunk.split(" ")) > 1:
        print(noun_chunk)

river thames
major settlement
south east
united kingdom
great britain
most populous city
two millennia


### Классификация

In [44]:
% matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 6.0)
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.datasets.samples_generator import make_blobs

In [45]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

Билиотека sklearn позволяет легко загрузить коллекцию документов 20-newsgroups, часто используемую для сравнения методов классификации и кластеризации текстов. Загрузим документы из двух категорий:

In [46]:
print("Loading 20 newsgroups dataset...")

dataset = fetch_20newsgroups(subset='all', categories=['talk.religion.misc', 'talk.politics.misc'],
                             shuffle=True, random_state=42)

print("%d documents" % len(dataset.data))
print("%d categories:" % len(dataset.target_names), dataset.target_names)

Loading 20 newsgroups dataset...
1403 documents
2 categories: ['talk.politics.misc', 'talk.religion.misc']


Каждый документ представлен сырым текстом (plain text). Например, так выглядит первый документ, и он про политику:

In [47]:
print(dataset.data[3])

From: bskendig@netcom.com (Brian Kendig)
Subject: Re: Is it good that Jesus died?
Organization: Starfleet Headquarters: San Francisco
Lines: 15

sandvik@newton.apple.com (Kent Sandvik) writes:
>
>I've done all those things, and I've regretted it, and I learned 
>a lesson or two. So far an aspirin, a good talk with your wife,
>or a one week vacation has cured me -- no need for group therapy
>or strange religions!

Um, Kent... just what *have* you been doing with his wife?!?  ;-D

-- 
_/_/_/  Brian Kendig                             Je ne suis fait comme aucun
/_/_/  bskendig@netcom.com                de ceux que j'ai vus; j'ose croire
_/_/                            n'etre fait comme aucun de ceux qui existent.
  /  The meaning of life     Si je ne vaux pas mieux, au moins je suis autre.
 /    is that it ends.                                           -- Rousseau



In [48]:
print(dataset.target_names[dataset.target[0]])

talk.politics.misc


Классы сбалансированы:

In [49]:
Y = np.array(dataset.target)
print(np.bincount(Y))

[775 628]


Чтобы обучать модели классификации, нам нужно сначала описать каждый документ каким-то набором признаков. Будем считать, что каждый документ - это мешок слов (порядок слов не важен), подсчитаем частоту каждого слова в документе (Term frequency) и оштрафуем общеупотребительные слова с помощью обратной документной частоты (Inverted document frequency). Такой подход называется TfIdf и часто используется на практике для работы с текстами.

Реализация в sklearn позволяет также указать параметры фильтрации словаря. В данном случае мы выбрасываем слова, которые встречаются более чем в 70% документов, менее чем в 10 документах, а также английские стоп-слова.

In [50]:
vectorizer = TfidfVectorizer(max_df=0.7, min_df=10, stop_words='english')
X = vectorizer.fit_transform(dataset.data)

Получаем матрицу объект-признак. Каждый документ описывается важностью слов (tf-idf скором), ставших признаками:

In [51]:
X.shape

(1403, 3854)

Теперь можно учить модели! 

### Построение модели и оценка качества

In [52]:
from sklearn import cross_validation
from sklearn import metrics

Важно: чтобы иметь возможность честно измерить качество модели, разобьем выборку на обучающую и контрольную. На обучении натренируем SVM, а не контроле измерим точность классификации (accuracy). Для этого есть удобный метод:

In [53]:
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(X, Y, test_size=0.1, random_state=0)

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(1262, 3854) (1262,)
(141, 3854) (141,)


In [54]:
svm = SGDClassifier(loss="hinge", n_iter=200).fit(X_train, Y_train)
svm.score(X_test, Y_test)

/Users/anastasiazuhba/anaconda3/envs/Py3ML/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


0.9716312056737588

Подсчет accuracy другим способом:

In [55]:
Y_pred = svm.predict(X_test)
metrics.accuracy_score(Y_test, Y_pred)

0.9716312056737588

Аналогичным образом в sklearn.metrics реализовано множество других полезных показателей:

In [56]:
print('Recall:', metrics.recall_score(Y_test, Y_pred))
print('Precision:', metrics.precision_score(Y_test, Y_pred))
print('F1-score:', metrics.f1_score(Y_test, Y_pred))
print('ROC_AUC:', metrics.roc_auc_score(Y_test, Y_pred))

Recall: 0.984375
Precision: 0.9545454545454546
F1-score: 0.9692307692307692
ROC_AUC: 0.9727069805194805


Получилось очень неплохое качество классификации! TfIdf + SVM является классическим бейзлайном для классификации текстов.

Что можно сделать дальше:
* Сделать кроссвалидацию и подобрать гиперпараметры.
* Посчитать confusion-matrix http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html 
* Нарисовать ROC-кривые, например, так: http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
* Обучить многоклассовый SVM (в оригинальном датасете 20 категорий, а не 2).

### Логистическая регрессия для той же задачи



In [57]:
from sklearn.linear_model import LogisticRegression

In [58]:
from sklearn import grid_search

In [59]:
param_grid = {"C": [0.001, 0.01, 0.1, 1, 10, 100, 1000], "penalty": ["l1", "l2"]}
model = LogisticRegression()
cv = cross_validation.KFold(len(Y_train), n_folds=10, shuffle=True, random_state=1234)
gs = grid_search.GridSearchCV(model, param_grid, scoring="roc_auc", cv=cv, verbose=10)
gs.fit(X_train, Y_train)

print("Best score is: ", gs.best_score_)
print("Best parametrs:")

best_params = gs.best_estimator_.get_params()

for param_name in sorted(best_params.keys()):
    print(param_name, ":", best_params[param_name])

Fitting 10 folds for each of 14 candidates, totalling 140 fits
[CV] C=0.001, penalty=l1 .............................................
[CV] .................... C=0.001, penalty=l1, score=0.500000 -   0.0s
[CV] C=0.001, penalty=l1 .............................................
[CV] .................... C=0.001, penalty=l1, score=0.500000 -   0.0s
[CV] C=0.001, penalty=l1 .............................................
[CV] .................... C=0.001, penalty=l1, score=0.500000 -   0.0s
[CV] C=0.001, penalty=l1 .............................................
[CV] .................... C=0.001, penalty=l1, score=0.500000 -   0.0s
[CV] C=0.001, penalty=l1 .............................................
[CV] .................... C=0.001, penalty=l1, score=0.500000 -   0.0s
[CV] C=0.001, penalty=l1 .............................................
[CV] .................... C=0.001, penalty=l1, score=0.500000 -   0.0s
[CV] C=0.001, penalty=l1 .............................................
[CV] .........

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s remaining:    0.0s


[CV] C=0.001, penalty=l2 .............................................
[CV] .................... C=0.001, penalty=l2, score=0.976562 -   0.0s
[CV] C=0.001, penalty=l2 .............................................
[CV] .................... C=0.001, penalty=l2, score=0.993878 -   0.0s
[CV] C=0.001, penalty=l2 .............................................
[CV] .................... C=0.001, penalty=l2, score=0.990385 -   0.0s
[CV] C=0.001, penalty=l2 .............................................
[CV] .................... C=0.001, penalty=l2, score=0.994898 -   0.0s
[CV] C=0.001, penalty=l2 .............................................
[CV] .................... C=0.001, penalty=l2, score=0.992411 -   0.0s
[CV] C=0.001, penalty=l2 .............................................
[CV] .................... C=0.001, penalty=l2, score=0.969075 -   0.0s
[CV] C=0.01, penalty=l1 ..............................................
[CV] ..................... C=0.01, penalty=l1, score=0.500000 -   0.0s
[CV] C

[CV] ........................ C=1, penalty=l2, score=0.997217 -   0.0s
[CV] C=1, penalty=l2 .................................................
[CV] ........................ C=1, penalty=l2, score=0.984407 -   0.0s
[CV] C=10, penalty=l1 ................................................
[CV] ....................... C=10, penalty=l1, score=0.995455 -   0.0s
[CV] C=10, penalty=l1 ................................................
[CV] ....................... C=10, penalty=l1, score=0.998470 -   0.0s
[CV] C=10, penalty=l1 ................................................
[CV] ....................... C=10, penalty=l1, score=0.982359 -   0.0s
[CV] C=10, penalty=l1 ................................................
[CV] ....................... C=10, penalty=l1, score=0.977991 -   0.0s
[CV] C=10, penalty=l1 ................................................
[CV] ....................... C=10, penalty=l1, score=0.968750 -   0.0s
[CV] C=10, penalty=l1 ................................................
[CV] .

[CV] ..................... C=1000, penalty=l2, score=0.999745 -   0.1s
[CV] C=1000, penalty=l2 ..............................................
[CV] ..................... C=1000, penalty=l2, score=0.996964 -   0.1s
[CV] C=1000, penalty=l2 ..............................................
[CV] ..................... C=1000, penalty=l2, score=0.993503 -   0.2s
Best score is:  0.9960365617070364
Best parametrs:
C : 1000
class_weight : None
dual : False
fit_intercept : True
intercept_scaling : 1
max_iter : 100
multi_class : ovr
n_jobs : 1
penalty : l2
random_state : None
solver : liblinear
tol : 0.0001
verbose : 0
warm_start : False


[Parallel(n_jobs=1)]: Done 140 out of 140 | elapsed:    4.9s finished


In [60]:
lr = LogisticRegression(C=1000, penalty='l2').fit(X_train, Y_train)
Y_lr = lr.predict(X_test)
metrics.accuracy_score(Y_test, Y_lr)

0.9645390070921985